In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

TODO:

when running with cloud. 
if you dont need to keep the execution logs, change the cloud settings to keep it for a short while
Check that the cache if you need to set the cache to false (especially in plot process)

In [2]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [3]:
# %%bash
# export GOOGLE_APPLICATION_CREDENTIALS="/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json"
# gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS


In [4]:
!gcloud auth list
# !gcloud config set account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [5]:
# in the command line. based on:
# https://nextflow.io/blog/2023/nextflow-with-gbatch.html
# gcloud services enable batch.googleapis.com compute.googleapis.com logging.googleapis.com
# gcloud iam service-accounts create eila-batch
# gcloud iam service-accounts list 


# gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/storage.admin"


In [6]:
import os
# os.environ["TOWER_ACCESS_TOKEN"] = 'XXX'
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [7]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [8]:
# !chmod +x ./bin/scrna_qc_metrics.py
import os
bin_fld = os.path.join(os.getcwd(),"bin")
bin_fld

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin'

In [9]:
!chmod +x {os.path.join(bin_fld, "joint_cell_plotting.py")}
!chmod +x {os.path.join(bin_fld, "joint_cell_plotting_density.R")}
!chmod +x {os.path.join(bin_fld, "write_html.py")}



In [10]:
# !nextflow -self-update

In [11]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [12]:
nf_data_base_path = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/'

# Write the configuration file
with open("nextflow_vm.config", "w") as config_file:
    config_file.write(f"""
manifest {{
  description = 'A nexflow pipeline for fg single cell joint qc' 
  mainScript = 'nf_wf_fg_singlecell_joint.nf'
}}

profiles {{
  google_batch_profile {{
    workDir = 'gs://eila-nextflow-bucket/work/ATAC'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = false
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }}   
}}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_PY='joint_cell_plotting.py'
params.SC_JOINT_CELL_BARCODE_PLOT_RNA_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na.na'
params.SC_JOINT_CELL_BARCODE_PLOT_ATAC_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na1.na'
//Minimum number of UMIs/fragments required for a cell to be plotted
params.SC_JOINT_CELL_BARCODE_PLOT_REMOVE_LOW_YIELDING_CELLS=5
params.SC_JOINT_CELL_BARCODE_PLOT_METADATA_FILE='{nf_data_base_path}nf_wf_fg_singlecell/na2.na'
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_UMIS=5
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_GENES=5
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_TSS=5
//Cutoff for minimum number of ATAC fragments
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_FRAGS=5
params.SC_JOINT_CELL_BARCODE_PLOT_PKR='PKR123'
params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_R='joint_cell_plotting_density.R'

params.SC_JOINT_CELL_BARCODE_PLOT_HTML_SCRIPT='write_html.py'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_ATAC_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na.na'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_RNA_METRICS='{nf_data_base_path}nf_wf_fg_singlecell/na1.na'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_IMAGE_FILES='{nf_data_base_path}nf_wf_fg_singlecell/na2.na'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_LOG_FILES='{nf_data_base_path}nf_wf_fg_singlecell/na3.na'
""")


In [13]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [14]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/work
# !gsutil rm -r gs://eila-nextflow-bucket/work/ATAC

In [15]:
# !rm .nextflow*

In [16]:
# !ls nextflow.config

In [17]:
!gcloud config list


[compute]
region = us-central1
zone = us-central1-a
[core]
account = eila-batch@hai-gcp-genomic.iam.gserviceaccount.com
disable_usage_reporting = True
project = hai-gcp-genomic

Your active configuration is: [default]


In [ ]:
!nextflow run nf_wf_fg_singlecell_joint.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile


# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_joint.nf` [pensive_leavitt] DSL2 - revision: d926c7e73f
Dec 18, 2023 3:36:55 AM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
Before calling run_joint_barcode_cell_plotting
RNA Metrics: /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na.na
ATAC Metrics: /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na1.na
Uploading local `bin` scripts folder to gs://eila-nextflow-bucket/work/ATAC/tmp/73/13d52149fa20847c9af7aa02649f27/bin
[-        ] process > run_joint_barcode_cell_plot... -
[-        ] process > run_joint_cell_plotting_den... -
[-        ] process > run_joint_metrics_html_report  -
After calling run_joint_barcode_cell_plotting
Before calling run_joint_cell_plotting_density
After calling run_joint_cell_plotting_density
Before run_joint_metrics_html_report
Before run_joint_metrics_html_report
After run_joint_

In [ ]:
!cat .nextflow.log